# Welcome to My Loan Default Project 
Dataset from Kaggle :https://www.kaggle.com/datasets/mishra5001/credit-card

- Exploratory analysis to understand the data set, clean the data set, and identify key column for the analysis
- Perform univariate, bivariate, and multivariate analysis for target categorical and numerical variables
- Identifying the variables which can help in predicting high risk customers 

Assumtioons:เราไม่ทราบว่าลูกค้าพลาดการชำระเงินกี่ครั้ง ลูกค้าทุกคนที่พลาดการชำระเงินอย่างน้อยหนึ่งครั้งจะได้รับการรับมือเหมือนกันเหมือนกัน


#read file

# รานชื่อสมาชิก

 - ปิยวัช ขันทนิเทศน์ 6504053620109
 - สุชาครีย์ ศีลาเจริญ 6504053620125
 - จิรภาส สามารถรัมย์ 6504053630023
 - สุธิภัทร จำเริญดี 6504053630171
 - นายกำภูกฤช ศรีวิเศษ 6504053630015


In [0]:
app_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/s6504053620109@email.kmutnb.ac.th/application_data-4.csv")

#import libary we need

In [0]:
import pyspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import matplotlib.pyplot as plt

import pandas as pd
import  warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from pyspark.sql.functions import col, count, when, isnan

In [0]:
'''pip install xgboost pyspark'''

In [0]:
import pickle

# EDA

## 1. Reading the Input Data from File

In [0]:
'"app_df.display()"'

In [0]:
app_df.dtypes

## Datatype issue we need to convert

In [0]:

# กำหนดคอลัมน์ที่ควรเป็น float และ int
int_columns = ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
               'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
               'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
               'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
               'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2',
               'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7',
               'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
               'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
               'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']

float_columns = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
                 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG',
                 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG',
                 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
                 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
                 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',
                 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
                 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
                 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',
                 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
                 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
                 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
                 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
                 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

# แปลงคอลัมน์ที่ควรเป็น float
for column in float_columns:
    app_df = app_df.withColumn(column, col(column).cast("float"))

# แปลงคอลัมน์ที่ควรเป็น int
for column in int_columns:
    app_df = app_df.withColumn(column, col(column).cast("int"))

# ตรวจสอบ schema หลังจากการแปลง
app_df.printSchema()


## 2. Data Quality Check And Missing Values

## 2.1 Percentage of missing values for columns and rows

In [0]:
total_count = app_df.count()

missing_percentage_df = (app_df.select([(count(when(col(c).isNull() | isnan(c), c)) / total_count * 100).alias(c) for c in app_df.columns])
                         .toPandas().T.rename(columns={0: 'percentage_missing_value'})
                         .sort_values(by='percentage_missing_value', ascending=False))

print(missing_percentage_df)


In [0]:
#สรุปเปอร์เซ็นต์ของค่าที่ขาด
no_missing = (100.0 * missing_percentage_df[missing_percentage_df['percentage_missing_value'] == 0].count() / len(missing_percentage_df)).round(2)
low_missing = (100.0 * missing_percentage_df[(missing_percentage_df['percentage_missing_value'] > 0) & (missing_percentage_df['percentage_missing_value'] <= 10)].count() / len(missing_percentage_df)).round(2)
mid_missing = (100.0 * missing_percentage_df[(missing_percentage_df['percentage_missing_value'] > 10) & (missing_percentage_df['percentage_missing_value'] <= 50)].count() / len(missing_percentage_df)).round(2)
high_missing = (100.0 * missing_percentage_df[missing_percentage_df['percentage_missing_value'] > 50].count() / len(missing_percentage_df)).round(2)

# Print the summary
print(f"{no_missing[0]}% columns have no missing value")
print(f"{low_missing[0]}% columns have missing value between 0-10%")
print(f"{mid_missing[0]}% columns have missing value between 10-50%")
print(f"{high_missing[0]}% columns have more than 50% missing value")


- คอลัมน์ที่ไม่มีค่า missing (0% missing): จำนวนและเปอร์เซ็นต์ของคอลัมน์ที่ไม่มีค่าขาดหายไปเลย (มีข้อมูลครบถ้วนทั้งหมด)
- คอลัมน์ที่มีค่า missing ในช่วง 0-10%: จำนวนและเปอร์เซ็นต์ของคอลัมน์ที่มีค่าขาดหายไปเล็กน้อย (อยู่ระหว่าง 0-10%)
- คอลัมน์ที่มีค่า missing ในช่วง 10-50%: จำนวนและเปอร์เซ็นต์ของคอลัมน์ที่มีค่าขาดหายไปปานกลาง (อยู่ระหว่าง 10-50%)
- คอลัมน์ที่มีค่า missing มากกว่า 50%: จำนวนและเปอร์เซ็นต์ของคอลัมน์ที่มีค่าขาดหายไปมากกว่า 50% ซึ่งแสดงถึงการขาดหายไปของข้อมูลในระดับสูง

### checking row-wise null percentages

In [0]:
# checking row-wise null percentages

# สร้างคอลัมน์ใหม่ 'num_missing_value' เพื่อเก็บจำนวนค่าที่ขาดหายไปในแต่ละแถว
row_null = app_df.withColumn(
    'num_missing_value', 
    sum(when(col(c).isNull(), 1).otherwise(0) for c in app_df.columns)
)

# แสดงผลคอลัมน์ 'num_missing_value'
row_null.select('num_missing_value').show()

## 2.2 Removing the columns with high percentage of missing values(>50%)

In [0]:
# จำนวนแถวทั้งหมดใน DataFrame
total_count = app_df.count()

# คำนวณเปอร์เซ็นต์ของค่าที่ขาดหายไปในแต่ละคอลัมน์
cols_to_drop = [c for c in app_df.columns if (app_df.filter(col(c).isNull()).count() / total_count * 100) > 50]

# ลบคอลัมน์ที่มีค่าที่ขาดหายไปมากกว่า 50%
app_df_1 = app_df.drop(*cols_to_drop)

# แสดงคอลัมน์ที่เหลือ
print(app_df_1.columns)

### re-checking columns with missing values


In [0]:
# จำนวนแถวทั้งหมดใน DataFrame
total_count = app_df_1.count()

# คำนวณเปอร์เซ็นต์ของค่าที่ขาดหายไปในแต่ละคอลัมน์ แล้วเรียงลำดับ
missing_percentage = (app_df_1.select([(count(when(col(c).isNull() | isnan(c), c)) / total_count * 100).alias(c) for c in app_df_1.columns])
                      .toPandas().T.rename(columns={0: 'percentage_missing_value'})
                      .sort_values(by='percentage_missing_value', ascending=True))

# แสดงผล
print(missing_percentage)

## 2.3 Checking the data type of columns and fixing the incorrect data types

In [0]:
#checking the data types to identify the incorrect data type
app_df_1.dtypes

### We did not find any column with incorrect data types. Each categorical column is of type object and each numerical column is either of float or int type

In [0]:
from pyspark.sql.functions import col, abs as pyspark_abs, round as pyspark_round

# แปลงหน่วยจากวันเป็นปี (โดยการหารด้วย 365) และแปลงเป็นค่าบวก
app_df_1 = app_df.withColumn('DAYS_BIRTH', pyspark_round(pyspark_abs(col('DAYS_BIRTH') / 365), 2)) \
                 .withColumn('DAYS_EMPLOYED', pyspark_round(pyspark_abs(col('DAYS_EMPLOYED') / 365), 2)) \
                 .withColumn('DAYS_REGISTRATION', pyspark_round(pyspark_abs(col('DAYS_REGISTRATION') / 365), 2)) \
                 .withColumn('DAYS_ID_PUBLISH', pyspark_round(pyspark_abs(col('DAYS_ID_PUBLISH') / 365), 2)) \
                 .withColumn('DAYS_LAST_PHONE_CHANGE', pyspark_round(pyspark_abs(col('DAYS_LAST_PHONE_CHANGE') / 365), 2))

# แสดงผลลัพธ์ของ DataFrame ที่แก้ไขแล้ว
app_df_1.display()


## 2.4 Identifying Outliers for Numerical columns

### For the outlier analysis of numerical columns, we will focus on
- AMT_GOODS_PRICE
- AMT_INCOME_TOTAL
- AMT_CREDIT
- AMT_ANNUITY
- FLOORSMAX_AVG

#### 2.4.1 Outlier analysis for AMT_GOODS_PRICE

In [0]:
app_df_1_pandas = app_df_1.select('AMT_GOODS_PRICE').toPandas()

sns.boxplot(app_df_1_pandas['AMT_GOODS_PRICE'] / 1000.0)
plt.xlabel('AMT_GOODS_PRICE (in 000s)')


In [0]:
# คำนวณสถิติสำหรับคอลัมน์ AMT_GOODS_PRICE โดยแบ่งค่าด้วย 1000
app_df_1.select((col('AMT_GOODS_PRICE') / 1000).alias('AMT_GOODS_PRICE_in_1000')).describe().show()


In [0]:
# คำนวณควอร์ไทล์ที่ 1 (25%) และควอร์ไทล์ที่ 3 (75%)
quantiles = app_df_1.approxQuantile('AMT_GOODS_PRICE', [0.25, 0.75], 0.05)

# คำนวณ IQR (Interquartile Range)
IQR_AMT_GOODS_PRICE = (quantiles[1] - quantiles[0]) / 1000

# คำนวณ Upper Limit สำหรับ boxplot
Upper_limit_IQR_AMT_GOODS_PRICE = (quantiles[1] / 1000) + IQR_AMT_GOODS_PRICE * 1.5

Upper_limit_IQR_AMT_GOODS_PRICE


In [0]:
from pyspark.sql.functions import col

# กรองแถวที่มีค่า AMT_GOODS_PRICE มากกว่า Upper_limit_IQR_AMT_GOODS_PRICE
outliers_count = app_df_1.filter((col('AMT_GOODS_PRICE') / 1000) > Upper_limit_IQR_AMT_GOODS_PRICE).count()

# จำนวนแถวทั้งหมดใน DataFrame
total_count = app_df_1.count()

# คำนวณเปอร์เซ็นต์ของค่าผิดปกติ (outliers)
percentage_outliers = round(100.0 * outliers_count / total_count, 2)

# แสดงผลลัพธ์
percentage_outliers


 From the boxplot, we can see there are outliers in AMT_GOODS_PRICE. The percentage of outliers stands at 4.83%

#### Outlier   ของ AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "FLOORSMAX_AVG

In [0]:
# แปลง PySpark DataFrame เป็น Pandas DataFrame
app_df_1_pandas = app_df_1.select("AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "FLOORSMAX_AVG").toPandas()

# สร้าง boxplot สำหรับแต่ละคอลัมน์
plt.figure(figsize=(12, 8))

# สร้าง subplot สำหรับแต่ละคอลัมน์
plt.subplot(2, 2, 1)
sns.boxplot(y=app_df_1_pandas['AMT_INCOME_TOTAL'] / 1000)
plt.title('AMT_INCOME_TOTAL (in 000s)')

plt.subplot(2, 2, 2)
sns.boxplot(y=app_df_1_pandas['AMT_CREDIT'] / 1000)
plt.title('AMT_CREDIT (in 000s)')

plt.subplot(2, 2, 3)
sns.boxplot(y=app_df_1_pandas['AMT_ANNUITY'] / 1000)
plt.title('AMT_ANNUITY (in 000s)')

plt.subplot(2, 2, 4)
sns.boxplot(y=app_df_1_pandas['FLOORSMAX_AVG'])
plt.title('FLOORSMAX_AVG')

# แสดงผล
plt.tight_layout()
plt.show()

In [0]:
# ฟังก์ชันคำนวณ IQR และ Upper Limit สำหรับแต่ละคอลัมน์
def calculate_upper_limit(df, column):
    quantiles = df.approxQuantile(column, [0.25, 0.75], 0.05)  # คำนวณ Q1 และ Q3
    IQR = quantiles[1] - quantiles[0]  # คำนวณ IQR
    upper_limit = quantiles[1] + 1.5 * IQR  # คำนวณ Upper Limit
    return upper_limit

# คำนวณ Upper Limit สำหรับแต่ละคอลัมน์
upper_limit_income = calculate_upper_limit(app_df_1, 'AMT_INCOME_TOTAL')
upper_limit_credit = calculate_upper_limit(app_df_1, 'AMT_CREDIT')
upper_limit_annuity = calculate_upper_limit(app_df_1, 'AMT_ANNUITY')
upper_limit_floorsmax = calculate_upper_limit(app_df_1, 'FLOORSMAX_AVG')


In [0]:
# ฟังก์ชันคำนวณจำนวนและเปอร์เซ็นต์ของ outliers
def calculate_outliers_percentage(df, column, upper_limit):
    outliers_count = df.filter(col(column) > upper_limit).count()  # กรองค่าที่เป็น outliers
    total_count = df.count()  # จำนวนแถวทั้งหมด
    percentage_outliers = round(100.0 * outliers_count / total_count, 2)  # คำนวณเปอร์เซ็นต์
    return percentage_outliers

# คำนวณเปอร์เซ็นต์ของ outliers สำหรับแต่ละคอลัมน์
outliers_income = calculate_outliers_percentage(app_df_1, 'AMT_INCOME_TOTAL', upper_limit_income)
outliers_credit = calculate_outliers_percentage(app_df_1, 'AMT_CREDIT', upper_limit_credit)
outliers_annuity = calculate_outliers_percentage(app_df_1, 'AMT_ANNUITY', upper_limit_annuity)
outliers_floorsmax = calculate_outliers_percentage(app_df_1, 'FLOORSMAX_AVG', upper_limit_floorsmax)

# แสดงผลลัพธ์
print(f"Outliers percentage for AMT_INCOME_TOTAL: {outliers_income}%")
print(f"Outliers percentage for AMT_CREDIT: {outliers_credit}%")
print(f"Outliers percentage for AMT_ANNUITY: {outliers_annuity}%")
print(f"Outliers percentage for FLOORSMAX_AVG: {outliers_floorsmax}%")


- AMT_INCOME_TOTAL: มีค่าผิดปกติ 4.56% ซึ่งแสดงถึงกลุ่มข้อมูลรายได้ที่ค่อนข้างสูงหรือต่ำผิดปกติเมื่อเทียบกับข้อมูลส่วนใหญ่
- AMT_CREDIT: มีค่าผิดปกติ 3.55% ซึ่งบ่งชี้ว่ามีจำนวนเล็กน้อยของผู้กู้ที่มีวงเงินสินเชื่อที่ผิดปกติเมื่อเทียบกับคนอื่น ๆ
- AMT_ANNUITY: มีค่าผิดปกติ 3.61% ซึ่งหมายถึงจำนวนเงินงวดการชำระต่อปีที่มีค่าแตกต่างจากกลุ่มข้อมูลหลัก
- FLOORSMAX_AVG: มีค่าผิดปกติ 1.7% ซึ่งบ่งบอกถึงจำนวนชั้นสูงสุดของอาคารที่แตกต่างจากกลุ่มข้อมูลทั่วไป

## Binning of continuous Variable

 For the binning, we will use following columns
- AGE_GROUP
- AMT_CATEGORY

In [0]:
app_df_1.select("DAYS_BIRTH").display()

In [0]:
from pyspark.sql.functions import when, col

# แบ่งช่วงอายุ (AGE_GROUP) โดยตรงจากค่าของ DAYS_BIRTH (ที่เป็นจำนวนปีอยู่แล้ว)
app_df_1 = app_df_1.withColumn(
    "AGE_GROUP", 
    when(col("DAYS_BIRTH").between(0, 19), "10s")
    .when(col("DAYS_BIRTH").between(20, 29), "20s")
    .when(col("DAYS_BIRTH").between(30, 39), "30s")
    .when(col("DAYS_BIRTH").between(40, 49), "40s")
    .when(col("DAYS_BIRTH").between(50, 59), "50s")
    .when(col("DAYS_BIRTH").between(60, 69), "60s")
    .when(col("DAYS_BIRTH").between(70, 79), "70s")
    .when(col("DAYS_BIRTH").between(80, 89), "80s")
    .otherwise("Unknown")
)

# คำนวณจำนวนแถวในแต่ละกลุ่มอายุ และเรียงลำดับจากมากไปน้อย
age_group_counts = app_df_1.groupBy("AGE_GROUP").count().orderBy(col("count").desc())

# แสดงผลลัพธ์
age_group_counts.show()


In [0]:
from pyspark.sql.functions import when, col

# แบ่งช่วงของ AMT_INCOME_TOTAL และสร้างคอลัมน์ใหม่ AMT_CATEGORY
app_df_1 = app_df_1.withColumn(
    "AMT_CATEGORY", 
    when(col("AMT_INCOME_TOTAL").between(0, 100000), "Low")
    .when(col("AMT_INCOME_TOTAL").between(100001, 200000), "Average")
    .when(col("AMT_INCOME_TOTAL").between(200001, 300000), "Good")
    .when(col("AMT_INCOME_TOTAL").between(300001, 400000), "Best")
    .when(col("AMT_INCOME_TOTAL").between(400001, 500000), "High")
    .when(col("AMT_INCOME_TOTAL").between(500001, 600000), "Very High")
    .otherwise("Unknown")
)

# แสดงผลลัพธ์
app_df_1.select("AMT_INCOME_TOTAL", "AMT_CATEGORY").show(5)


## 3. Analysis

ในการวิเคราะห์เพิ่มเติม เราจะลบคอลัมน์ที่ไม่เกี่ยวข้องออก และดำเนินการวิเคราะห์ต่อด้วยคอลัมน์ที่เลือกไว้เพียงไม่กี่คอลัมน์

In [0]:
# list of columns to be dropped
drop_columns = ['FLAG_CONT_MOBILE',
                'FLAG_MOBIL',
                'FLAG_EMP_PHONE',
                'FLAG_WORK_PHONE',
                'FLAG_PHONE',
                'FLAG_EMAIL',
                'HOUR_APPR_PROCESS_START',
                'WEEKDAY_APPR_PROCESS_START',
                'FLOORSMAX_AVG',
                'EXT_SOURCE_2',
                'EXT_SOURCE_3',
                'FLOORSMAX_AVG',
                'FLOORSMAX_MODE',
                'FLOORSMAX_MEDI',
                'TOTALAREA_MODE',
                'EMERGENCYSTATE_MODE',
                'REGION_POPULATION_RELATIVE',
                'YEARS_BEGINEXPLUATATION_AVG',
                'YEARS_BEGINEXPLUATATION_MEDI',
                'YEARS_BEGINEXPLUATATION_MODE',
                'REG_REGION_NOT_LIVE_REGION',
                'REG_REGION_NOT_WORK_REGION',
                'LIVE_REGION_NOT_WORK_REGION',
                'REG_CITY_NOT_LIVE_CITY',
                'REG_CITY_NOT_WORK_CITY',
                'LIVE_CITY_NOT_WORK_CITY',
                'FLAG_DOCUMENT_2',
                'FLAG_DOCUMENT_3',
                'FLAG_DOCUMENT_4',
                'FLAG_DOCUMENT_5',
                'FLAG_DOCUMENT_6',
                'FLAG_DOCUMENT_7',
                'FLAG_DOCUMENT_8',
                'FLAG_DOCUMENT_9',
                'FLAG_DOCUMENT_10',
                'FLAG_DOCUMENT_11',
                'FLAG_DOCUMENT_12',
                'FLAG_DOCUMENT_13',
                'FLAG_DOCUMENT_14',
                'FLAG_DOCUMENT_15',
                'FLAG_DOCUMENT_16',
                'FLAG_DOCUMENT_17',
                'FLAG_DOCUMENT_18',
                'FLAG_DOCUMENT_19',
                'FLAG_DOCUMENT_20',
                'FLAG_DOCUMENT_21'
               ]

In [0]:
app_df_2 = app_df_1.drop(*drop_columns)

In [0]:
app_df_2.display(5)

In [0]:
total_count = app_df_2.count()

# คำนวณเปอร์เซ็นต์ของค่าที่ขาดหายไปและเรียงลำดับ
missing_percentage_df = (app_df_2.select([(count(when(col(c).isNull() | isnan(c), c)) / total_count * 100).alias(c) for c in app_df_2.columns])
                         .toPandas().T.rename(columns={0: 'percentage_missing_value'})
                         .sort_values(by='percentage_missing_value'))

# แสดงผลลัพธ์
print(missing_percentage_df)


## 3.1 Checking imbalance in data

In [0]:
# คำนวณจำนวนคนที่ไม่มีหนี้สินคงค้าง (TARGET == 0)
target_0_count = app_df_2.filter(col("TARGET") == 0).count()

# คำนวณจำนวนคนที่มีหนี้สินคงค้าง (TARGET == 1)
target_1_count = app_df_2.filter(col("TARGET") == 1).count()

# คำนวณจำนวนทั้งหมดใน DataFrame
total_count = app_df_2.count()

# คำนวณเปอร์เซ็นต์ของ TARGET == 0 และ TARGET == 1
target_0_percentage = round((target_0_count / total_count) * 100, 2)
target_1_percentage = round((target_1_count / total_count) * 100, 2)

# แสดงผลลัพธ์
print(f"Target_0_percentage: {target_0_percentage}%")
print(f"Target_1_percentage: {target_1_percentage}%")


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col

# คำนวณจำนวนคนที่ไม่มีหนี้สินคงค้าง (TARGET == 0)
target_0_count = app_df_2.filter(col("TARGET") == 0).count()

# คำนวณจำนวนคนที่มีหนี้สินคงค้าง (TARGET == 1)
target_1_count = app_df_2.filter(col("TARGET") == 1).count()

# สร้างข้อมูลสำหรับการ plot
data = {
    'Target': ['No Default (TARGET == 0)', 'Default (TARGET == 1)'],
    'Count': [target_0_count, target_1_count]
}

# สร้าง DataFrame สำหรับการ visualize
import pandas as pd
df = pd.DataFrame(data)

# สร้างกราฟ bar plot สำหรับจำนวนของ TARGET == 0 และ TARGET == 1
plt.figure(figsize=(8, 6))
sns.barplot(x='Target', y='Count', data=df)
plt.title('Count of No Default vs Default')
plt.ylabel('Count')
plt.xlabel('Target')
plt.show()


### 3.1.1 Creating Target_0_df

In [0]:
# กรองข้อมูลที่ TARGET == 0 (non-defaulters)
target_0_df = app_df_2.filter(col("TARGET") == 0)

# แสดงผลลัพธ์
target_0_df.show(5)


### Creating Target_1_df

In [0]:
# กรองข้อมูลที่ TARGET == 0 (non-defaulters)
target_1_df = app_df_2.filter(col("TARGET") == 1)

# แสดงผลลัพธ์
target_1_df.display(5)


### Defind cat or num cols

In [0]:
#list of all categorical columns
categorical_columns = ['NAME_CONTRACT_TYPE',
                       'FLAG_OWN_CAR',
                       'FLAG_OWN_REALTY',
                       'CODE_GENDER',
                       'NAME_EDUCATION_TYPE',
                       'AMT_CATEGORY',
                       'AGE_GROUP',
                       'NAME_FAMILY_STATUS',
                       'NAME_HOUSING_TYPE',
                       'NAME_TYPE_SUITE',
                       'NAME_INCOME_TYPE',
                       'OCCUPATION_TYPE',
                       'ORGANIZATION_TYPE',                       
                       'REGION_RATING_CLIENT_W_CITY',
                       'REGION_RATING_CLIENT',
                       'AMT_REQ_CREDIT_BUREAU_HOUR',
                       'DEF_60_CNT_SOCIAL_CIRCLE',
                       'AMT_REQ_CREDIT_BUREAU_WEEK',
                       'AMT_REQ_CREDIT_BUREAU_DAY',
                       'DEF_30_CNT_SOCIAL_CIRCLE',
                       'AMT_REQ_CREDIT_BUREAU_QRT',
                       'CNT_CHILDREN',
                       'CNT_FAM_MEMBERS',
                       'AMT_REQ_CREDIT_BUREAU_MON',
                       'AMT_REQ_CREDIT_BUREAU_YEAR',
                       'OBS_30_CNT_SOCIAL_CIRCLE',
                       'OBS_60_CNT_SOCIAL_CIRCLE',
                      ]
                      

In [0]:
# list of all continuous numerical column
numerical_columns= ['AMT_GOODS_PRICE',
                    'DAYS_LAST_PHONE_CHANGE',
                    'DAYS_ID_PUBLISH',
                    'AMT_INCOME_TOTAL',
                    'DAYS_EMPLOYED',
                    'DAYS_REGISTRATION',
                    'DAYS_BIRTH',
                    'AMT_CREDIT',
                    'AMT_ANNUITY'
                   ]

## 3.2 Univariate Analysis for categorical variable

In [0]:
# Loop สำหรับการวิเคราะห์ univariate analysis ของคอลัมน์เชิงหมวดหมู่
for i in categorical_columns:
    # สำหรับ TARGET = 0 (non-defaulters)
    target_0_df_pandas = target_0_df.groupBy(i).agg({'*': 'count'}).toPandas()
    # แทนที่ค่าที่เป็น None ด้วย 'Unknown' หรือค่าอื่นที่เหมาะสม
    target_0_df_pandas[i] = target_0_df_pandas[i].fillna('Unknown')
    target_0_df_pandas['percentage'] = target_0_df_pandas['count(1)'] / target_0_df.count() * 100

    # สำหรับ TARGET = 1 (defaulters)
    target_1_df_pandas = target_1_df.groupBy(i).agg({'*': 'count'}).toPandas()
    # แทนที่ค่าที่เป็น None ด้วย 'Unknown' หรือค่าอื่นที่เหมาะสม
    target_1_df_pandas[i] = target_1_df_pandas[i].fillna('Unknown')
    target_1_df_pandas['percentage'] = target_1_df_pandas['count(1)'] / target_1_df.count() * 100

    plt.figure(figsize=(12, 8))

    # Plot สำหรับ TARGET = 0
    plt.subplot(1, 2, 1)
    plt.bar(target_0_df_pandas[i], target_0_df_pandas['percentage'])
    plt.title(i + ' - Target = 0')
    plt.xticks(rotation=45)

    # Plot สำหรับ TARGET = 1
    plt.subplot(1, 2, 2)
    plt.bar(target_1_df_pandas[i], target_1_df_pandas['percentage'])
    plt.title(i + ' - Target = 1')
    plt.xticks(rotation=45)

    # ปรับการจัด layout
    plt.tight_layout()
    plt.show()


#### Key interpretation from univariate analysis of Categorical variables
  

- Code_Gender: ลูกค้าที่เป็นผู้ชายมีเปอร์เซ็นต์ของผู้ที่ผิดนัดชำระหนี้ (Target = 1) สูงกว่าผู้ที่ไม่ผิดนัดชำระหนี้ (Target = 0)
- NAME_EDUCATION_TYPE: ลูกค้าที่มีการศึกษาระดับมัธยมปลายหรือมัธยมศึกษาพิเศษ (Secondary/Secondat Special education) มีเปอร์เซ็นต์ของผู้ผิดนัดชำระหนี้ (Target = 1) สูงกว่ากลุ่มที่ไม่ผิดนัดชำระหนี้
- Age_Group: กลุ่มอายุที่มีเปอร์เซ็นต์ของผู้ผิดนัดชำระหนี้สูงที่สุดคือกลุ่มอายุ 30 ปี (30s)
- NAME_INCOME_TYPE: ลูกค้าที่มีอาชีพหลักคือการทำงานมีเปอร์เซ็นต์ของผู้ผิดนัดชำระหนี้สูงกว่ากลุ่มลูกค้าที่เป็นผู้รับบำนาญ ซึ่งกลุ่มลูกค้าผู้รับบำนาญมีอัตราการผิดนัดชำระหนี้ต่ำกว่าในกลุ่มลูกค้าที่ไม่ผิดนัดชำระหนี้
- OCCUPATION_TYPE: ลูกค้าที่ประกอบอาชีพกรรมกรมีเปอร์เซ็นต์ของการผิดนัดชำระหนี้ (Target = 1) สูงกว่าลูกค้าในกลุ่มที่ไม่ผิดนัดชำระหนี้
- REGION_RATING_CLIENT_W_CITY: ลูกค้าที่มีคะแนนระดับ 3 (จากระบบการจัดอันดับภูมิภาค) มีเปอร์เซ็นต์ของการผิดนัดชำระหนี้สูงกว่าลูกค้าในกลุ่มที่ไม่ผิดนัดชำระหนี้
- REGION_RATING_CLIENT: ลูกค้าที่มีคะแนนระดับ 3 มีเปอร์เซ็นต์ของการผิดนัดชำระหนี้สูงกว่ากลุ่มลูกค้าที่ไม่ผิดนัดชำระหนี้

## 3.3 Correlation for numerical columns

In [0]:


# สร้าง DataFrame ของคอลัมน์เชิงตัวเลข (numerical columns)
numerical_df = app_df_2.select(numerical_columns)

# คำนวณความสัมพันธ์สำหรับแต่ละคู่ของคอลัมน์เชิงตัวเลข
correlation_matrix = pd.DataFrame({col: [numerical_df.stat.corr(col, c) for c in numerical_columns] for col in numerical_columns}, index=numerical_columns)

# สร้าง heatmap โดยใช้ seaborn
plt.figure(figsize=(20,10))
sns.heatmap(correlation_matrix, annot=True)

# ปรับค่าขอบเขตของกราฟเพื่อให้แสดงผลอย่างถูกต้อง
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t)

plt.yticks(rotation=0)
plt.show()

In [0]:

# คำนวณ correlation matrix สำหรับ TARGET = 0
numerical_df_0 = target_0_df.select(numerical_columns)
correlation_matrix_0 = pd.DataFrame({col: [numerical_df_0.stat.corr(col, c) for c in numerical_columns] for col in numerical_columns}, index=numerical_columns)

# คำนวณ correlation matrix สำหรับ TARGET = 1
numerical_df_1 = target_1_df.select(numerical_columns)
correlation_matrix_1 = pd.DataFrame({col: [numerical_df_1.stat.corr(col, c) for c in numerical_columns] for col in numerical_columns}, index=numerical_columns)

# สร้าง heatmap สำหรับ TARGET = 0 และ TARGET = 1
plt.figure(figsize=(20, 8))

# Heatmap สำหรับ TARGET = 0
plt.subplot(1, 2, 1)
sns.heatmap(correlation_matrix_0, annot=True)
plt.title('heatmap - Target = 0')
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t)
plt.yticks(rotation=0)

# Heatmap สำหรับ TARGET = 1
plt.subplot(1, 2, 2)
sns.heatmap(correlation_matrix_1, annot=True)
plt.title('heatmap - Target = 1')
b, t = plt.ylim()
b += 0.5
t -= 0.5
plt.ylim(b, t)
plt.yticks(rotation=0)

# แสดงผล
plt.tight_layout()
plt.show()


## 3.4 Checking if Variables with highest coeffecient are same in both file

In [0]:


# คำนวณ correlation matrix สำหรับ TARGET = 0
numerical_df_0 = target_0_df.select(numerical_columns)
correlation_matrix_0 = pd.DataFrame({col: [numerical_df_0.stat.corr(col, c) for c in numerical_columns] for col in numerical_columns}, index=numerical_columns)

# คำนวณ correlation matrix สำหรับ TARGET = 1
numerical_df_1 = target_1_df.select(numerical_columns)
correlation_matrix_1 = pd.DataFrame({col: [numerical_df_1.stat.corr(col, c) for c in numerical_columns] for col in numerical_columns}, index=numerical_columns)

# หา correlation ที่มีค่าสูงสุดสำหรับ TARGET = 0
max_corr_0 = correlation_matrix_0.abs().unstack().sort_values(ascending=False).drop_duplicates().idxmax()

# หา correlation ที่มีค่าสูงสุดสำหรับ TARGET = 1
max_corr_1 = correlation_matrix_1.abs().unstack().sort_values(ascending=False).drop_duplicates().idxmax()

# แสดงผลลัพธ์
print(f"ตัวแปรที่มีค่าสัมประสิทธิ์สูงสุดใน TARGET = 0: {max_corr_0}")
print(f"ตัวแปรที่มีค่าสัมประสิทธิ์สูงสุดใน TARGET = 1: {max_corr_1}")

# ตรวจสอบว่าตัวแปรที่มีค่าสูงสุดเหมือนกันหรือไม่
if max_corr_0 == max_corr_1:
    print("ตัวแปรที่มีค่าสัมประสิทธิ์สูงสุดในทั้งสองกลุ่มเหมือนกัน")
else:
    print("ตัวแปรที่มีค่าสัมประสิทธิ์สูงสุดในทั้งสองกลุ่มแตกต่างกัน")


## 3.5 Univariate Analysis for Numerical Values

In [0]:

import matplotlib.pyplot as plt
import seaborn as sns

# Loop สำหรับการสร้าง boxplot สำหรับแต่ละคอลัมน์เชิงตัวเลข
for i in numerical_columns:
    # แปลง DataFrame เป็น Pandas DataFrame
    target_0_df_pandas = target_0_df.select(i).toPandas()
    target_1_df_pandas = target_1_df.select(i).toPandas()
    
    plt.figure(figsize=(20, 10))

    # Boxplot สำหรับ TARGET = 0
    plt.subplot(1, 2, 1)
    sns.boxplot(target_0_df_pandas[i])
    plt.title(i + ' - Target = 0')

    # Boxplot สำหรับ TARGET = 1
    plt.subplot(1, 2, 2)
    sns.boxplot(target_1_df_pandas[i])
    plt.title(i + ' - Target = 1')

    # แสดงผล
    plt.tight_layout()
    plt.show()


## The objective of this analysis is to find pattern in defaulter vs non-defaulter customers wrt target

In [0]:
# Loop สำหรับการวิเคราะห์ bivariate analysis ของตัวแปรเชิงตัวเลข
for i in numerical_columns:
    # แปลง PySpark DataFrame เป็น Pandas DataFrame ก่อน
    app_df_pandas = app_df_2.select('TARGET', i).toPandas()

    # สร้างกราฟ boxplot โดยใช้ Seaborn
    sns.boxplot(data=app_df_pandas, x='TARGET', y=i)
    
    # แสดงกราฟ
    plt.show()


การตีความที่สำคัญจากการวิเคราะห์เชิงเดี่ยวของตัวแปรเชิงตัวเลข
ในส่วนนี้ เราจะเน้นเฉพาะตัวแปรที่มีความแตกต่างอย่างมีนัยสำคัญระหว่างกลุ่ม target = 0 และ target = 1
- DAYS_LAST_PHONE_CHANGE: ค่ามัธยฐานและเปอร์เซ็นไทล์ที่ 75 ของกลุ่มผู้ผิดนัดชำระหนี้ (Target = 1) ต่ำกว่ากลุ่มที่ไม่ผิดนัดชำระหนี้  -   - ซึ่งหมายความว่าผู้ผิดนัดชำระหนี้มีแนวโน้มที่จะเปลี่ยนหมายเลขโทรศัพท์บ่อยขึ้นก่อนที่จะทำการสมัคร
- DAYS_ID_PUBLISH: ผู้ผิดนัดชำระหนี้ดูเหมือนจะเปลี่ยนรหัสประจำตัวบ่อยกว่าผู้ที่ไม่ผิดนัดชำระหนี้
- DAYS_BIRTH: ค่าเปอร์เซ็นไทล์ที่ 25 ค่ามัธยฐาน และเปอร์เซ็นไทล์ที่ 75 สำหรับอายุของผู้สมัครที่เป็นผู้ผิดนัดชำระหนี้น้อยกว่าผู้สมัครที่อายุน้อยกว่า ซึ่งหมายความว่ากลุ่มผู้ผิดนัดชำระหนี้มีแนวโน้มที่จะเป็นคนอายุน้อยกว่ากลุ่มที่ไม่ผิดนัดชำระหนี้







## 3.6 Bivariate Analysis for Categorical Variable

In [0]:
import matplotlib.pyplot as plt

# Loop สำหรับการสร้าง bar plot สำหรับแต่ละคอลัมน์เชิงหมวดหมู่
for i in categorical_columns:
    # คำนวณค่าเฉลี่ยของ TARGET สำหรับแต่ละค่าในคอลัมน์เชิงหมวดหมู่
    mean_target_df = app_df_2.groupBy(i).agg({'TARGET': 'mean'}).toPandas()

    # สร้าง bar plot
    mean_target_df.plot(x=i, y='avg(TARGET)', kind='bar', legend=False)
    plt.title(i + ' vs Target')
    plt.ylabel('Average Target')
    
    # แสดงผล
    plt.show()


### สรุป
- CODE_GENDER: ลูกค้าที่เป็นผู้ชายมีความน่าจะเป็นในการผิดนัดชำระหนี้สูงกว่า
- NAME_EDUCATION_TYPE: ลูกค้าที่มีการศึกษาระดับมัธยมศึกษาตอนต้นหรือต่ำกว่ามีความเสี่ยงต่อการผิดนัดชำระหนี้สูงกว่า
- AGE_GROUP: ลูกค้าที่อยู่ในกลุ่มอายุ 20 และ 30 ปีมีโอกาสผิดนัดชำระหนี้สูงกว่า
- NAME_HOUSING_TYPE: ลูกค้าที่อาศัยอยู่ในอพาร์ทเมนท์เช่าหรืออยู่กับพ่อแม่มีแนวโน้มที่จะผิดนัดชำระหนี้มากกว่า
- NAME_INCOME_TYPE: ลูกค้าที่ว่างงานและลูกค้าที่อยู่ในช่วงลาคลอดมีความเสี่ยงสูงกว่า
- OCCUPATION_TYPE: แรงงานทักษะต่ำมีแนวโน้มที่จะผิดนัดชำระหนี้มากกว่า
- REGION_RATING_CLIENT และ REGION_RATING_CLIENT_W_CITY: ลูกค้าที่มีคะแนนระดับ 3 มีความเสี่ยงในการผิดนัดชำระหนี้สูงกว่า

# train test split

In [0]:
from pyspark.sql import functions as F

# ตรวจสอบจำนวนค่า null ในแต่ละคอลัมน์ (ทั้งเชิงตัวเลขและหมวดหมู่)
app_df_2.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in numerical_columns + categorical_columns]).show()


### check null

In [0]:
mean_value = app_df_2.select(F.mean(F.col("DAYS_LAST_PHONE_CHANGE"))).collect()[0][0]
app_df_2 = app_df_2.na.fill({"DAYS_LAST_PHONE_CHANGE": mean_value})

# เติมค่า null ในคอลัมน์ OCCUPATION_TYPE ด้วย 'Unknown'
app_df_2 = app_df_2.na.fill({"OCCUPATION_TYPE": "Unknown"})

# ตรวจสอบผลลัพธ์หลังจากเติมค่า
app_df_2.select("OCCUPATION_TYPE").show(5)



In [0]:
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline

In [0]:
feature_col =categorical_columns +numerical_columns

In [0]:
all_columns = categorical_columns + numerical_columns

# แสดง DataFrame ที่รวมคอลัมน์ทั้งสองชุด
display(app_df_2.select(all_columns))

## Encoders And Create pipeline

In [0]:
from pyspark.sql import functions as F

# แปลงคอลัมน์เชิงตัวเลขเป็นชนิด double
app_df_2 = app_df_2.withColumn("DAYS_LAST_PHONE_CHANGE", F.col("DAYS_LAST_PHONE_CHANGE").cast("double"))
app_df_2 = app_df_2.withColumn("DAYS_ID_PUBLISH", F.col("DAYS_ID_PUBLISH").cast("double"))

# Categorical Columns
categorical_columns = [
    "CODE_GENDER", "NAME_EDUCATION_TYPE", "AGE_GROUP", 
    "NAME_HOUSING_TYPE", "NAME_INCOME_TYPE", "OCCUPATION_TYPE",
    "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY"
]

# Numerical Columns
numerical_columns = ["DAYS_LAST_PHONE_CHANGE", "DAYS_ID_PUBLISH"]

# 1. Label Encoding (ใช้ StringIndexer)
indexers = [StringIndexer(inputCol=col, outputCol=col + "_Indexed") for col in categorical_columns]

# 2. One-Hot Encoding
encoders = [OneHotEncoder(inputCol=col + "_Indexed", outputCol=col + "_Vector") for col in categorical_columns]

# 3. Scaling ข้อมูลเชิงตัวเลข
assembler = VectorAssembler(inputCols=numerical_columns, outputCol="numerical_features")
scaler = MinMaxScaler(inputCol="numerical_features", outputCol="scaled_numerical_features")

# 4. Combine Categorical and Numerical features
assembler_final = VectorAssembler(
    inputCols=[col + "_Vector" for col in categorical_columns] + ["scaled_numerical_features"], 
    outputCol="features"
)

# Apply transformations in stages
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler, assembler_final])
model = pipeline.fit(app_df_2)
final_df = model.transform(app_df_2)

# แสดงผลฟีเจอร์สุดท้าย
final_df.select("features").show(truncate=False)


## RandomForest Classifier

In [0]:
from pyspark.ml.classification import RandomForestClassifier

# เพิ่ม RandomForestClassifier
rf = RandomForestClassifier(labelCol="TARGET", featuresCol="features")

# ใช้ Pipeline เหมือนเดิม แต่เพิ่ม RandomForest เข้าไปในขั้นตอนสุดท้าย
pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler, assembler_final, rf])

# ฝึกโมเดล
model = pipeline.fit(app_df_2)

# ทำนายผล
predictions = model.transform(app_df_2)

# แสดงผลลัพธ์
predictions.select("features", "TARGET", "prediction").show()


###  AUC ROC BY Random forest

In [0]:
'''from pyspark.ml.evaluation import BinaryClassificationEvaluator
# ประเมินผลด้วย AUC-ROC
binary_evaluator = BinaryClassificationEvaluator(labelCol="TARGET", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = binary_evaluator.evaluate(predictions)
print(f"AUC-ROC: {roc_auc}")

In [0]:
# Make predictions using the model (ตัวอย่าง Logistic Regression)
predictions = lr_model.transform(final_df)

# หรือในกรณีของ Decision Tree:
# predictions = dt_model.transform(final_df)

# ประเมินผลด้วย AUC-ROC
binary_evaluator = BinaryClassificationEvaluator(labelCol="TARGET", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = binary_evaluator.evaluate(predictions)
print(f"AUC-ROC: {roc_auc}")

# ประเมิน Accuracy และ F1 Score
multiclass_evaluator = MulticlassClassificationEvaluator(labelCol="TARGET", predictionCol="prediction", metricName="accuracy")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="TARGET", predictionCol="prediction", metricName="f1")

accuracy = multiclass_evaluator.evaluate(predictions)
f1_score = f1_evaluator.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")


### Logistic regression

#### modeling

In [0]:
from pyspark.ml.classification import LogisticRegression

# Initialize Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="TARGET", maxIter=10)

# Train the model
lr_model = lr.fit(final_df)

# Make predictions
lr_predictions = lr_model.transform(final_df)

# Show predictions
lr_predictions.select("features", "TARGET", "prediction").show(truncate=False)


#### summary

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

binary_evaluator = BinaryClassificationEvaluator(labelCol="TARGET", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
multiclass_evaluator = MulticlassClassificationEvaluator(labelCol="TARGET", predictionCol="prediction", metricName="accuracy")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="TARGET", predictionCol="prediction", metricName="f1")
precision_evaluator = MulticlassClassificationEvaluator(labelCol="TARGET", predictionCol="prediction", metricName="precisionByLabel")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="TARGET", predictionCol="prediction", metricName="recallByLabel")

# Logistic Regression Metrics
print("Logistic Regression Metrics:")
lr_auc = binary_evaluator.evaluate(lr_predictions)
lr_accuracy = multiclass_evaluator.evaluate(lr_predictions)
lr_f1 = f1_evaluator.evaluate(lr_predictions)
lr_precision = precision_evaluator.evaluate(lr_predictions)
lr_recall = recall_evaluator.evaluate(lr_predictions)

print(f"AUC: {lr_auc}")
print(f"Accuracy: {lr_accuracy}")
print(f"F1 Score: {lr_f1}")
print(f"Precision: {lr_precision}")
print(f"Recall: {lr_recall}")

### Decision tree

#### modeling

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

# Initialize Decision Tree model
dt = DecisionTreeClassifier(featuresCol="features", labelCol="TARGET")

# Train the model
dt_model = dt.fit(final_df)

# Make predictions
dt_predictions = dt_model.transform(final_df)

# Show predictions
dt_predictions.select("features", "TARGET", "prediction").show(truncate=False)


#### Summary

In [0]:
print("Decision Tree Metrics:")
dt_auc = binary_evaluator.evaluate(dt_predictions)
dt_accuracy = multiclass_evaluator.evaluate(dt_predictions)
dt_f1 = f1_evaluator.evaluate(dt_predictions)
dt_precision = precision_evaluator.evaluate(dt_predictions)
dt_recall = recall_evaluator.evaluate(dt_predictions)

print(f"AUC: {dt_auc}")
print(f"Accuracy: {dt_accuracy}")
print(f"F1 Score: {dt_f1}")
print(f"Precision: {dt_precision}")
print(f"Recall: {dt_recall}")

### XG boost

In [0]:
pip show xgboost


#### modeling

In [0]:
import xgboost as xgb
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# แปลง Spark DataFrame เป็น Pandas DataFrame เพื่อใช้กับ XGBoost
final_pandas_df = final_df.select("features", "TARGET").toPandas()

# สร้าง DMatrix สำหรับ XGBoost
dtrain = xgb.DMatrix(data=final_pandas_df['features'].tolist(), label=final_pandas_df['TARGET'])

# สร้างโมเดล XGBoost
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "scale_pos_weight": 10,  # ใช้จัดการข้อมูลไม่สมดุล
    "max_depth": 5,
    "eta": 0.1,
    "num_round": 100
}

# ฝึกโมเดล XGBoost
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# ทำนายผล
predictions = xgb_model.predict(dtrain)

# แสดงผลลัพธ์
print(predictions)


#### ผลลัพธ์ 

In [0]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# ทำนายผล (หลังจากที่ฝึกโมเดลเสร็จ)
predictions_prob = xgb_model.predict(dtrain)  # ผลลัพธ์เป็นความน่าจะเป็น

# แปลงค่าความน่าจะเป็นเป็น class (0 หรือ 1)
threshold = 0.5
predictions_class = [1 if prob > threshold else 0 for prob in predictions_prob]

# ค่าเป้าหมาย (label) ที่แท้จริงจาก dtrain
y_true = final_pandas_df['TARGET']

# 1. คำนวณ AUC-ROC
auc = roc_auc_score(y_true, predictions_prob)
print(f"AUC-ROC: {auc}")

# 2. คำนวณ Accuracy
accuracy = accuracy_score(y_true, predictions_class)
print(f"Accuracy: {accuracy}")

# 3. คำนวณ F1-Score
f1 = f1_score(y_true, predictions_class)
print(f"F1-Score: {f1}")


#### Add scale_pos_weight For imbalance Data ,num_boost_round up to 1500

add Hyperparameter

In [0]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# คำนวณ scale_pos_weight จากข้อมูลที่ไม่สมดุล
num_negatives = len(final_pandas_df[final_pandas_df['TARGET'] == 0])
num_positives = len(final_pandas_df[final_pandas_df['TARGET'] == 1])
scale_pos_weight = num_negatives / num_positives

# สร้าง DMatrix สำหรับ XGBoost
dtrain = xgb.DMatrix(data=final_pandas_df['features'].tolist(), label=final_pandas_df['TARGET'])

# ปรับพารามิเตอร์ของ XGBoost สำหรับข้อมูลไม่สมดุลและข้อมูลขนาดใหญ่
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "scale_pos_weight": scale_pos_weight,  # ใช้เพื่อจัดการกับข้อมูลไม่สมดุล
    "max_depth": 7,            # ความลึกของต้นไม้
    "eta": 0.3,                # อัตราการเรียนรู้
    "n_estimators": 500,       # จำนวนต้นไม้
    "subsample": 0.8,          # สัดส่วนของข้อมูลที่ใช้สร้างต้นไม้
    "colsample_bytree": 0.8    # สัดส่วนของฟีเจอร์ที่สุ่มเลือกในการสร้างต้นไม้
}

# ฝึกโมเดล
xgb_model = xgb.train(params, dtrain, num_boost_round=1500)

# ทำนายผล
predictions_prob = xgb_model.predict(dtrain)

# แปลงค่าความน่าจะเป็นเป็น class
threshold = 0.5
predictions_class = [1 if prob > threshold else 0 for prob in predictions_prob]

# ค่าเป้าหมายที่แท้จริง
y_true = final_pandas_df['TARGET']

# คำนวณ AUC-ROC, Accuracy, และ F1-Score
auc = roc_auc_score(y_true, predictions_prob)
accuracy = accuracy_score(y_true, predictions_class)
f1 = f1_score(y_true, predictions_class)

# แสดงผลลัพธ์
print(f"AUC-ROC: {auc}")
print(f"Accuracy: {accuracy}")
print(f"F1-Score: {f1}")


## Save model

In [0]:
import pickle

# บันทึกโมเดล XGBoost ลงในโฟลเดอร์ชั่วคราว (/tmp/)
with open('/tmp/my_xgboost_model.pkl', 'wb') as model_file:
    pickle.dump(xgb_model, model_file)


In [0]:
%fs ls /tmp/

In [0]:
dbutils.fs.cp("file:/tmp/my_xgboost_model.pkl", "dbfs:/FileStore/my_xgboost_model.pkl")


In [0]:
%fs ls /FileStore/
